In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/large_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import MLP_Models, LSTM_Models, GRU_Models
from utils.benchmark import Benchmark

In [2]:
conf = Config()
models = [MLP_Models().mlp_4, LSTM_Models().lstm_4, GRU_Models().gru_4]
model_names = ["mlp_4", "lstm_4", "gru_4"]

In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "mlp")

mlp_dataset = NetworkDataset(X_test, y_test)
mlp_loader = DataLoader(mlp_dataset, conf.batch_size)

X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "lstm")

rnn_dataset = NetworkDataset(X_test, y_test)
rnn_loader = DataLoader(rnn_dataset, conf.batch_size)

In [4]:
for i in range(len(models)):
    # set parameters
    model = models[i]
    loader = rnn_loader if i > 0 else mlp_loader
    name = model_names[i]
    result_path = os.path.join(conf.benchmark_host, "large_model", name + ".txt")

    # run benchmark
    benchmark = Benchmark(model, loader, conf.batch_size, name, result_path)
    benchmark()

    # print and save result
    benchmark.print_result()
    benchmark.save()

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/large_model/mlp_4.pth!


[W1215 10:47:23.885558770 CPUAllocator.cpp:245] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event
ERROR:2025-12-15 10:47:23 2771988:2771988 DeviceProperties.cpp:47] gpuGetDeviceCount failed with code 35


Benchmark - mlp_4 model:

Memory usage (MB):
Avg memory usage: 0.155MB
Peak memory usage: 1.201MB

Model inference latency on one batch (batch size = 32):
Avg latency: 0.361ms
Min latency: 0.333ms
Max latency: 1.051ms

Model inference throughput (batch size = 32):
Throughput: 45650.34 samples/sec

Model inference CPU usage (number of logical cores) during runtime:
CPU runtime: 0.04 seconds
Average CPU usage: 48.89/96 cores

Model (mlp_4) inference accuracy (%):
Accuracy: 87.50%



Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/large_model/lstm_4.pth!
Benchmark - lstm_4 model:

Memory usage (MB):
Avg memory usage: 25.610MB
Peak memory usage: 323.977MB

Model inference latency on one batch (batch size = 32):
Avg latency: 417.203ms
Min latency: 340.288ms
Max latency: 460.423ms

Model inference throughput (batch size = 32):
Throughput: 74.44 samples/sec

Model inference CPU usage (number of logical cores) during runtime:
CPU runtime: 21.44 seconds
Averag